In [31]:
import pandas as pd
import numpy as np

In [32]:
df = pd.read_csv('vibemeter_dataset.csv')
df.head()

,Employee_ID,Response_Date,Vibe_Score,Emotion_Zone
0,EMP0454,2023-01-01,3,Sad Zone
1,EMP0071,2023-01-02,1,Leaning to Happy Zone
2,EMP0096,2023-01-03,4,Neutral Zone (OK)
3,EMP0206,2023-01-04,6,Neutral Zone (OK)
4,EMP0074,2023-01-05,3,Excited Zone


In [33]:
#sort based on employee id
df.sort_values(by=["Employee_ID", "Response_Date"], ascending=[True, False], inplace=True)
df.head()

,Employee_ID,Response_Date,Vibe_Score,Emotion_Zone
28,EMP0002,2023-01-29,1,Frustrated Zone
406,EMP0003,2024-02-11,3,Leaning to Sad Zone
363,EMP0003,2023-12-30,1,Leaning to Sad Zone
414,EMP0004,2024-02-19,4,Neutral Zone (OK)
343,EMP0005,2023-12-10,3,Frustrated Zone


In [34]:
# Change values of emotion zone by numbers
df['Emotion_Zone'] = df['Emotion_Zone'].replace(['Frustrated Zone'], 0)
df['Emotion_Zone'] = df['Emotion_Zone'].replace(['Sad Zone'], 1)
df['Emotion_Zone'] = df['Emotion_Zone'].replace(['Leaning to Sad Zone'], 2)
df['Emotion_Zone'] = df['Emotion_Zone'].replace(['Neutral Zone (OK)'], 3)
df['Emotion_Zone'] = df['Emotion_Zone'].replace(['Leaning to Happy Zone'], 4)
df['Emotion_Zone'] = df['Emotion_Zone'].replace(['Happy Zone'], 5)
df['Emotion_Zone'] = df['Emotion_Zone'].replace(['Excited Zone'], 6)
df.head()

/tmp/ipykernel_71945/201845183.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Emotion_Zone'] = df['Emotion_Zone'].replace(['Excited Zone'], 6)


,Employee_ID,Response_Date,Vibe_Score,Emotion_Zone
28,EMP0002,2023-01-29,1,0
406,EMP0003,2024-02-11,3,2
363,EMP0003,2023-12-30,1,2
414,EMP0004,2024-02-19,4,3
343,EMP0005,2023-12-10,3,0


In [35]:
#make a column days since recent and for multiple entries of same employee id, for the most recent entry (date) it will be 0, for the next recent entry it will be that date- recent date
df["Response_Date"] = pd.to_datetime(df["Response_Date"], errors='coerce')
df.sort_values(by=["Employee_ID", "Response_Date"], ascending=[True, False], inplace=True)


decay_factor = 1 / 5  

df["Days_Since"] = df.groupby("Employee_ID")["Response_Date"].transform(lambda x: (x.max() - x).dt.days)


df["Weight"] = np.exp(-decay_factor * df["Days_Since"])


df.head()



,Employee_ID,Response_Date,Vibe_Score,Emotion_Zone,Days_Since,Weight
28,EMP0002,2023-01-29,1,0,0,1.000000
406,EMP0003,2024-02-11,3,2,0,1.000000
363,EMP0003,2023-12-30,1,2,43,0.000184
414,EMP0004,2024-02-19,4,3,0,1.000000
343,EMP0005,2023-12-10,3,0,0,1.000000


In [36]:
# Compute weighted emotion score per employee
weighted_emotions = df.groupby("Employee_ID").apply(
    lambda emp: pd.Series({
        "Weighted_Emotion_Score": np.sum(emp["Emotion_Zone"] * emp["Weight"]) / np.sum(emp["Weight"])
    })
).reset_index()

# Display the final weighted emotion scores
weighted_emotions.head()


/tmp/ipykernel_71945/735707495.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weighted_emotions = df.groupby("Employee_ID").apply(


,Employee_ID,Weighted_Emotion_Score
0,EMP0002,0.000000e+00
1,EMP0003,2.000000e+00
2,EMP0004,3.000000e+00
3,EMP0005,2.343632e-20
4,EMP0006,5.000000e+00


In [37]:
# weighted emotion score to 4 decimal
weighted_emotions["Weighted_Emotion_Score"] = weighted_emotions["Weighted_Emotion_Score"].round(4)
weighted_emotions.head()

,Employee_ID,Weighted_Emotion_Score
0,EMP0002,0.0
1,EMP0003,2.0
2,EMP0004,3.0
3,EMP0005,0.0
4,EMP0006,5.0


In [39]:
output_path = "unique_employees_weighted_emotion.csv"
weighted_emotions.to_csv(output_path, index=False)

print(f"Processed data saved to {output_path}")


Processed data saved to unique_employees_weighted_emotion.csv
